# Лабораторная работа №2

## Задание 1

Провести классификацию найденного датасета, методом машины опорных векторов.

Импорт библиотек

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.svm import SVC

Загрузка датасета

In [2]:
df = pd.read_csv('weather2.csv', encoding='utf-8')
df['RainToday']= df['RainToday'].map({'Yes': True, 'No': False}).astype(bool) 
df['RainTomorrow']= df['RainTomorrow'].map({'Yes': True, 'No': False}).astype(bool)
print(df.head(5))
print(df.dtypes)

         Date Location  MinTemp  MaxTemp  Rainfall  Evaporation  Sunshine  \
0  2008-12-01   Albury     13.4     22.9       0.6          NaN       NaN   
1  2008-12-02   Albury      7.4     25.1       0.0          NaN       NaN   
2  2008-12-03   Albury     12.9     25.7       0.0          NaN       NaN   
3  2008-12-04   Albury      9.2     28.0       0.0          NaN       NaN   
4  2008-12-05   Albury     17.5     32.3       1.0          NaN       NaN   

  WindGustDir  WindGustSpeed WindDir9am  ... Humidity9am  Humidity3pm  \
0           W           44.0          W  ...        71.0         22.0   
1         WNW           44.0        NNW  ...        44.0         25.0   
2         WSW           46.0          W  ...        38.0         30.0   
3          NE           24.0         SE  ...        45.0         16.0   
4           W           41.0        ENE  ...        82.0         33.0   

   Pressure9am  Pressure3pm  Cloud9am  Cloud3pm  Temp9am  Temp3pm  RainToday  \
0       1007.7    

Удаление ненужных столбцов из датасета

In [3]:
columns_to_drop = ['Humidity3pm', 'Pressure3pm', 'Cloud3pm', 'Temp3pm', 'WindDir9am']
df = df.drop(columns=columns_to_drop, axis=1)
print(df.head(5))

         Date Location  MinTemp  MaxTemp  Rainfall  Evaporation  Sunshine  \
0  2008-12-01   Albury     13.4     22.9       0.6          NaN       NaN   
1  2008-12-02   Albury      7.4     25.1       0.0          NaN       NaN   
2  2008-12-03   Albury     12.9     25.7       0.0          NaN       NaN   
3  2008-12-04   Albury      9.2     28.0       0.0          NaN       NaN   
4  2008-12-05   Albury     17.5     32.3       1.0          NaN       NaN   

  WindGustDir  WindGustSpeed WindDir3pm  WindSpeed9am  WindSpeed3pm  \
0           W           44.0        WNW          20.0          24.0   
1         WNW           44.0        WSW           4.0          22.0   
2         WSW           46.0        WSW          19.0          26.0   
3          NE           24.0          E          11.0           9.0   
4           W           41.0         NW           7.0          20.0   

   Humidity9am  Pressure9am  Cloud9am  Temp9am  RainToday  RainTomorrow  
0         71.0       1007.7       8.

Удаление строк с пропущенными значениями

In [4]:
df.dropna(inplace=True)

Предобработка данных

In [5]:
# Кодирование категориальных признаков
label_encoders = {}
for column in df.select_dtypes(include=['object']).columns:
    label_encoders[column] = LabelEncoder()
    df[column] = label_encoders[column].fit_transform(df[column])
print(df.head(5))
print(df.dtypes)

      Date  Location  MinTemp  MaxTemp  Rainfall  Evaporation  Sunshine  \
6049   418         3     17.9     35.2       0.0         12.0      12.3   
6050   419         3     18.4     28.9       0.0         14.8      13.0   
6052   421         3     19.4     37.6       0.0         10.8      10.6   
6053   422         3     21.9     38.4       0.0         11.4      12.2   
6054   423         3     24.2     41.0       0.0         11.2       8.4   

      WindGustDir  WindGustSpeed  WindDir3pm  WindSpeed9am  WindSpeed3pm  \
6049           11           48.0          12           6.0          20.0   
6050            8           37.0          10          19.0          19.0   
6052            5           46.0           6          30.0          15.0   
6053           14           31.0          15           6.0           6.0   
6054           14           35.0          14          17.0          13.0   

      Humidity9am  Pressure9am  Cloud9am  Temp9am  RainToday  RainTomorrow  
6049         20

In [6]:
# Масштабирование числовых признаков
scaler = StandardScaler()
numeric_features = df.select_dtypes(include=['int32', 'int64', 'float32', 'float64']).columns
df[numeric_features] = scaler.fit_transform(df[numeric_features])
print(df.head(5))

          Date  Location   MinTemp   MaxTemp  Rainfall  Evaporation  Sunshine  \
6049 -1.509900 -1.115357  0.741099  1.798783 -0.305009     1.984401  1.280727   
6050 -1.508714 -1.115357  0.821053  0.828408 -0.305009     2.799929  1.465244   
6052 -1.506343 -1.115357  0.980961  2.168449 -0.305009     1.634889  0.832615   
6053 -1.505157 -1.115357  1.380730  2.291671 -0.305009     1.809645  1.254368   
6054 -1.503971 -1.115357  1.748517  2.692143 -0.305009     1.751393  0.252705   

      WindGustDir  WindGustSpeed  WindDir3pm  WindSpeed9am  WindSpeed3pm  \
6049     0.751503       0.572852    1.028664     -1.036278      0.020757   
6050     0.124547      -0.231091    0.602206      0.432761     -0.091638   
6052    -0.502409       0.426680   -0.250710      1.675793     -0.541215   
6053     1.378460      -0.669606    1.668352     -1.036278     -1.552765   
6054     1.378460      -0.377263    1.455123      0.206755     -0.766004   

      Humidity9am  Pressure9am  Cloud9am   Temp9am  Rain

Разделение данных на обучающий и тестовый наборы

In [7]:
X = df.drop('RainTomorrow', axis=1)
Y = df['RainTomorrow']
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

Поиск оптимальных параметров

In [8]:
svm_model = SVC()
parameters = {'kernel':('rbf', 'linear', 'sigmoid'), 'C':[0.1, 0.3, 1], 'gamma':[1, 10, 100]}
grid_search = GridSearchCV(svm_model, parameters, cv=3, verbose=3,n_jobs=1)  # cv - количество фолдов для кросс-валидации
grid_search.fit(X_train, Y_train)

Fitting 3 folds for each of 27 candidates, totalling 81 fits
[CV 1/3] END ........C=0.1, gamma=1, kernel=rbf;, score=0.772 total time=  25.7s
[CV 2/3] END ........C=0.1, gamma=1, kernel=rbf;, score=0.772 total time=  25.4s
[CV 3/3] END ........C=0.1, gamma=1, kernel=rbf;, score=0.772 total time=  26.1s
[CV 1/3] END .....C=0.1, gamma=1, kernel=linear;, score=0.831 total time=   3.8s
[CV 2/3] END .....C=0.1, gamma=1, kernel=linear;, score=0.826 total time=   4.1s
[CV 3/3] END .....C=0.1, gamma=1, kernel=linear;, score=0.828 total time=   4.3s
[CV 1/3] END ....C=0.1, gamma=1, kernel=sigmoid;, score=0.704 total time=  11.0s
[CV 2/3] END ....C=0.1, gamma=1, kernel=sigmoid;, score=0.707 total time=   6.7s
[CV 3/3] END ....C=0.1, gamma=1, kernel=sigmoid;, score=0.711 total time=  11.5s
[CV 1/3] END .......C=0.1, gamma=10, kernel=rbf;, score=0.772 total time=  37.1s
[CV 2/3] END .......C=0.1, gamma=10, kernel=rbf;, score=0.772 total time=  34.8s
[CV 3/3] END .......C=0.1, gamma=10, kernel=rbf;

GridSearchCV(cv=3, estimator=SVC(), n_jobs=1,
             param_grid={'C': [0.1, 0.3, 1], 'gamma': [1, 10, 100],
                         'kernel': ('rbf', 'linear', 'sigmoid')},
             verbose=3)

Получение лучших параметров

In [9]:
best_params = grid_search.best_params_
print("Лучшие параметры:", best_params)

Лучшие параметры: {'C': 0.3, 'gamma': 1, 'kernel': 'linear'}


Обучение с лучшими параметрами

In [10]:
best_svm = grid_search.best_estimator_
best_svm.fit(X_train, Y_train)
# Обучение модели на тренировочных данных
svm_model.fit(X_train, Y_train)

SVC()

Предсказание на тестовом наборе

In [11]:
Y_pred = svm_model.predict(X_test)

Оценка производительности

In [12]:
accuracy = accuracy_score(Y_test, Y_pred)
report = classification_report(Y_test, Y_pred)

print(accuracy)
print(report)

0.8234172387490465
              precision    recall  f1-score   support

       False       0.84      0.95      0.89      5940
        True       0.75      0.42      0.54      1926

    accuracy                           0.82      7866
   macro avg       0.79      0.69      0.72      7866
weighted avg       0.81      0.82      0.80      7866



Перекрестная проверка для оценки модели

In [13]:
cv_scores = cross_val_score(svm_model, X_train, Y_train, cv=5)

print(cv_scores)
print(cv_scores.mean())

[0.83012871 0.83902749 0.83582327 0.82422123 0.83693579]
0.8332272995484645
